# Function Calling
* AKA tool calling 
* Function calling is a way to connect an AI model with external tools.
    * With all models, there are limitations to real-time information and risks of hallucination when prompted with complex questions. 
* Function calling allows you to connect your own functions within the chat completion API call by using the “tools” and “tool choice” parameters. 
    * tool_choice: Directs the model on which (if any) function to choose.
        * “none” or not defining this parameter makes the model not call a function and acts as a normal chat completion API call
        * “auto” makes the model decide on which function to call based on the context of the prompt
        * For a specific function call, you have to pass the function name in a dictionary format. This will force the model to choose that function.
        { "type": "function", "function": { "name": "my_function" }}
    * tools: When “tool choice” is set to “auto” or a specific function name, the model will refer to the “tools” parameter.
        * This holds the list of available tools to choose from.
        * To list the tools properly for the model to make the proper understanding of the function and its arguments, the tool list is formatted in a dictionary with natural language description of the overall function and each argument with its relationship to the function. 


If you haven't ran these pip install functions yet, run this code block.
If you've already installed these libraries, there's no need to run this again.

In [43]:
%pip install python-dotenv
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Import the libaries and load in the API key
* This is the same thing as we did for the chat completions API
* This is a way to safely access the API key and not have it hard coded into the script
* Notice that we also import the 'json' library. We will be using that on the response object

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import os, json

# I have my API key stored in a file called api.env
# The file is in the same directory as this script

# If the print statement returns None, then the API key is not being read
# Try using the full path to the file, and switching the slashes to forward slashes

load_dotenv("api.env") 
print(os.getenv("OPENAI_API_KEY"))

# Set up the API client and create the message list
* The API client is basically the portal into the available API services/data
* We use the API key as essentially a password to gain access to the client

In [60]:
# Create the connection to the OpenAI API using the API key
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

# Create a prompt for the user to enter a message
prompt = input("Enter a message: ")

# Create a list of messages to send to the API
message_list = [
    {
        "role": "system", 
        "content": "You are a helpful assistant."
    },
    {
        "role": "user", 
        "content": prompt
    }
]

# Create functions and a list of function descriptions
* We create the 'def' functions we want the model to be able to call
* Then we create a list of dictionaries which are natural language descriptions of the function and its arguments 

## Dictionary Schema for 'tools' parameter
    * type: The type of the tool. Currently only 'function' is supported.
    * function: A dictionary object that holds the details of the function
        * name: The name of the function. Has to be the same name as the corresponding 'def' function.
        * description: A description of what the function does.
        * parameters: A dictionary object that holds the details of the arguments
            * type: Will always be "object"
            * properties: A dictionary object that holds the details of each argument
                * name_of_the_argument: A dictionary object that holds details of this specific argument
                    * type: The type of data that this argument will be. I believe this only supports 'string' or 'object'.
                    * description: A description of what this argument is and how it relates to the function it is part of. It's always helpful to give an example.
                    * enum: You can create a list for the model to choose from if there are specific values you are looking for.
            * required: A list of arguments that are required. This allows for us to have arguments listed that don't need to be called if they aren't specified in the user prompt. 


In [61]:
# This is the actual function that the model's response object will call
# Notice how unit is an optional parameter (already initialized to "fahrenheit") 
def get_current_weather(location, unit="fahrenheit"):
    # Usually there would be more code here to get the weather data
    # But for now, we'll just return a set string saying that it is 80 degrees (possibly) fahrenheit at every location you give it
    return f"The current weather in {location} is 80 degrees {unit}."

# This is the list of dictionaries holding the natural language description of the function and the function's arguments
# This object has a specific schema (look in the notes above this codeblock) that needs to be followed 
tool_options = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA", # Notice how we give the model an example of the string that it should generate for this argument
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"] # Enum allows us to give a list of options the model can choose from instead of generating it's own value
                    },
                },
                "required": ["location"], # We set 'location' to be required and not 'unit' because unit is an optional argument that is initialized to 'fahrenheit' and the function will still properly run without it in the arguments.
            },
        }
    }
]

In [62]:
# An example of a tool_option object that holds multiple functions in it
# Notice how the last function doesn't have any arguments
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string", 
                        "enum": ["fahrenheit", "celcius"]
                    },
                },
                "required": ["location"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ask_wolfram",
            "description": "Ask Wolfram Alpha a question for factual information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The question to ask Wolfram Alpha.",
                    },
                },
                "required": ["query"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_time_and_date",
            "description": "Get the current time and date.",
        }
    }
]

# Make the API call
* This is the chat completions API still, but we are adding the two parameters that allow it make function calls.
* model and messages are required parameters in the chat completions API
* tool_choice: Can only be one of three options ('none', 'auto', { "type": "function", "function": { "name": "your_function_name" }})
    * I set this one to 'auto' with only one function detailed in 'tool_options' so that the model can choose whether to respond in natural language or with a function call.
    * 'none' will make it so that it will only respond in natural language, and with a dictionary like the one above, the model will always respond with that function call and its predicted arguments.
* tools: Takes the list of dictionaries object that holds the natural language description of the functions for the model to know what it can choose from.

In [63]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=message_list,
    tool_choice="auto",
    tools=tools,
    # temperature=0.0
)

# Navigate the API response object

In [66]:
# This is the whole response object from the API
# print(completion)
# Then we look at the choices that the model made
print(completion.choices)
# # We look at the message details that the model created
print(completion.choices[0].message)
# # We look at the natural language response from the model (this will be 'None' if it makes a function call)
print(completion.choices[0].message.content)
# # We look at the tool calls that the model decided to make
print(completion.choices[0].message.tool_calls)
# # We look at the first tool call and the function details 
print(completion.choices[0].message.tool_calls[0].function)
# # We look at the function name that the model picked out of the list of function descriptions it was provided (tool_options)
print(completion.choices[0].message.tool_calls[0].function.name)
# # We look at the function's arguments that the model chose. This is formatted as a JSON but is currently a string.
print(completion.choices[0].message.tool_calls[0].function.arguments)

# # We use the 'json' library we imported at the start to conver the string into an actual JSON object which we can then navigate like a dictionary.
arguments = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)
print(arguments["location"])

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PKUWdlVpzy3XqEA05QlQ6TUB', function=Function(arguments='{"location":"Miami","unit":"fahrenheit"}', name='get_current_weather'), type='function')]))]
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_PKUWdlVpzy3XqEA05QlQ6TUB', function=Function(arguments='{"location":"Miami","unit":"fahrenheit"}', name='get_current_weather'), type='function')])
None
[ChatCompletionMessageToolCall(id='call_PKUWdlVpzy3XqEA05QlQ6TUB', function=Function(arguments='{"location":"Miami","unit":"fahrenheit"}', name='get_current_weather'), type='function')]
Function(arguments='{"location":"Miami","unit":"fahrenheit"}', name='get_current_weather')
get_current_weather
{"location":"Miami","unit":"fahrenheit"}
Miami


Lets write some code that will take an API response object and do one of two things.
1. If the model responds with natural language, we display to the user using the print statement (easy)
2. If the model responds with a function call, we get the function details and run that function in our script (not as easy)

Note: It is helpful to make variables for certain sections of the response object so that way the code is a bit easier to read and comprehend

In [67]:
# Make variables for the natural language response or the tool call object that the model will generate
answer = completion.choices[0].message.content
tool_calls = completion.choices[0].message.tool_calls

# This code works because OpenAI's models respond with EITHER a natural language response or a function call. There are other models that can generate both in the same response object. 

# If tool_calls isn't empty (for natural language responses, it will be 'None')
if tool_calls:
    # For every tool call in the tool calls object (there usually is only one, but there could be more with parallel function calling)
    for tool_call in tool_calls:
        # Finds the function called and the arguments passed to the function from the API response
        function_called = tool_call.function.name
        function_args = tool_call.function.arguments
        # Converts the arguments to a JSON object instead of a JSON string
        function_args_json = json.loads(function_args)
        # Calls the function with the arguments
        # globals() is a Python native function that returns a dictionary with the global variables (including functions)
        if function_called in globals():
            # We call the function using eval which is another Python native function that "evaluates" based off it's knowledge of the global and local variables
            # (**function_args_json) is a way to pass the JSON object as arguments 
            # result holds the returned value of the function that was ran
            result = eval(f"{function_called}(**function_args_json)")
        # If there is no function by that name in the script, it will return an error message. Double check that the function name in the script and in the tool_options object are the same.
        else:
            result = "Function not found. Please try again."
        # This prints either the returned value of the successful function call or the error message saying it couldn't find the function in the script
        print(result)
# If tool_calls is empty, that means we just print out the natural language response from the model using the variable we created at the top
else:
    print(answer)

The current weather in Miami is 80 degrees fahrenheit.


In [40]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "get_ipython().run_line_magic('pip', 'install python-dotenv')\nget_ipython().run_line_magic('pip', 'install openai')",
  'from dotenv import load_dotenv\nfrom openai import OpenAI\nimport os, json\n\nload_dotenv("api.env")  # could pass in the path of the .env file in the arguments\nprint(os.getenv("OPENAI_API_KEY"))',
  'def get_current_weather(location, unit="fahrenheit"):\n    return f"The current weather in {location} is 80 degrees {unit}."\n\ntool_options = [\n    {\n        "type": "function",\n        "function": {\n            "name": "get_current_weather",\n            "description": "Get the current weather in a given location",\n            "parameters": {\n                "type": "object",\n         

## Here is an OpenAI example of getting a natural language response with the data returned from the function call

In [69]:
from openai import OpenAI
import json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        print(messages)
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response.choices[0].message.content
print(run_conversation())

[{'role': 'user', 'content': "What's the weather like in San Francisco, Tokyo, and Paris?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_3SpE7bNfEjdpod1ZD3Yn2nwU', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_kdLEgRyixQfiilh3IWJH2oAo', function=Function(arguments='{"location": "Tokyo, Japan"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_KroazvQ6VeLJLd1tFXE9DeC4', function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_3SpE7bNfEjdpod1ZD3Yn2nwU', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "San Francisco", "temperature": "72", "unit": null}'}, {'tool_call_id': 'call_kdLEgRyixQfiilh3IWJH2oAo', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"locati